In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
mimiciv_dir = 'C:/Users/suzie/Dropbox (MIT)/Spring 2021/6.871 Machine Learning for Healthcare/mimic-iv-1.0/hosp'

In [3]:
mimiciv_dir = 'data'

In [26]:
mimiciv_dir = './mimic_iv/hosp'

## Part I: read data

In [12]:
df_event = pd.read_csv(f'{mimiciv_dir}/diagnoses_icd.csv', low_memory=False)

In [13]:
df_code = pd.read_csv(f'{mimiciv_dir}/d_icd_diagnoses.csv', low_memory=False)

In [41]:

#merge tables:
df_sub = pd.merge(df_event, df_code, on = ['icd_code','icd_version'], how='left')

#convert to lower case:
df_sub['long_title'] = df_sub['long_title'].str.lower()

df_sub = df_sub.dropna(subset=['icd_code','icd_version','hadm_id','subject_id','long_title'], how='any')

# df_sub.head(5)

In [15]:
df_emar = pd.read_csv(f'{mimiciv_dir}/emar.csv', low_memory=False)
df_emar.head()

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,charttime,medication,event_txt,scheduletime,storetime
0,10000473,NaN,10000473-15,15,10000473-23,NaN,2138-03-16 03:53:00,Doxycycline Hyclate,Administered,2138-03-16 03:53:00,2138-03-16 03:53:00
1,10000764,NaN,10000764-17,17,10000764-9,NaN,2132-10-14 21:49:00,Amoxicillin-Clavulanic Acid,Administered,2132-10-14 21:49:00,2132-10-14 21:53:00
2,10000764,NaN,10000764-18,18,10000764-8,NaN,2132-10-14 21:49:00,Tetanus-DiphTox-Acellular Pertuss (Adacel),Administered,2132-10-14 21:49:00,2132-10-14 21:53:00
3,10000764,27897940.0,10000764-19,19,10000764-11,NaN,2132-10-14 23:54:00,PNEUMOcoccal 23-valent polysaccharide vaccine,Not Given,2132-10-14 23:54:00,2132-10-14 23:55:00
4,10000764,27897940.0,10000764-20,20,10000764-10,NaN,2132-10-14 23:55:00,Influenza Vaccine Quadrivalent,Not Given,2132-10-14 23:55:00,2132-10-14 23:55:00


In [16]:
df_emar_detail = pd.read_csv(f'{mimiciv_dir}/emar_detail.csv', low_memory=False)


,subject_id,emar_id,emar_seq,parent_field_ordinal,administration_type,pharmacy_id,barcode_type,reason_for_no_barcode,complete_dose_not_given,dose_due,...,infusion_rate_unit,route,infusion_complete,completion_interval,new_iv_bag_hung,continued_infusion_in_other_location,restart_interval,side,site,non_formulary_visual_verification
0,10000032,10000032-10,10,1.1,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,10000032-10,10,1.2,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,10000032-10,10,1.3,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,10000032-10,10,1.4,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,10000032-10,10,NaN,Standard Maintenance Medication,NaN,NaN,NaN,No,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
print(df_emar_detail.drop_duplicates(subset=["emar_id"]).shape[0])
print(df_emar_detail.drop_duplicates(subset=["emar_id", "route"]).shape[0])

27464367
35656372


## Part II: ICD Code Cohorting

In [42]:
#Filter to heart failure: 
#Filter 1: long title contains "heart failure" AND long title doesnt contain without heart failure
# this paper: https://academic.oup.com/aje/article/183/5/462/2462157?login=true used almost all of these for ADHF
df_f1 = df_sub[df_sub['long_title'].str.contains('heart failure')]
df_f1 = df_f1[~df_f1['long_title'].str.contains('without heart failure')]
# df_f1.head(5)

In [43]:
df_f1_hadm = df_f1[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
print(np.shape(df_f1_hadm)[0])
n1 = np.shape(df_f1_hadm)[0]
# df_f1_hadm.head(5)

56053


In [44]:
# Filter 2: Filter 1 + exclude "XX disease with heart failure" format (only keep HF as main target in title)
df_f2 = df_f1[~df_f1['long_title'].str.contains('disease')]
df_f2_hadm = df_f2[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
print(np.shape(df_f2_hadm)[0])
hf2_icd = df_f2[['long_title', 'icd_code','icd_version']].drop_duplicates(subset=['icd_code']) #.sort_values('icd_code')
hf2_icd
n2 = np.shape(df_f2_hadm)[0]

55935


In [45]:
# Filter 5: Filter 1+2+ Use I50+ and 428+
# source: https://icd.codes/icd10cm/I503 "The ICD code I50 is used to code Acute decompensated heart failure"
df_f5 = df_f2[df_f2['icd_code'].str.contains('428') | df_f2['icd_code'].str.contains('I50')]
df_f5_hadm = df_f5[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n5=(np.shape(df_f5_hadm)[0])
n5

55857

In [46]:
# Filter 6: Filter 1+2+5+ exclude acute only, exclude chronic only, keep acute on chronic or specified
# source: NA_CARE_Prov_ICD_10_CM_Documentation_and_Coding_Best_Practices_Heart_Failure_v3_ENG_2019 pdf
df_f6 = df_f5[~df_f5['long_title'].str.contains('acute')  | df_f5['long_title'].str.contains('acute on chronic') ]
df_f6 = df_f6[~df_f6['long_title'].str.contains('chronic')  | df_f6['long_title'].str.contains('acute on chronic') ]
df_f6_hadm = df_f6[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n6 = (np.shape(df_f6_hadm)[0])
n6

42892

In [47]:
# Filter 7: Filter 1+2+5+6 exclude end stage
# It is important to distinguish between AdHF and end-stage HF
# source https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6963179/
df_f7 = df_f6[~df_f6['long_title'].str.contains('end stage')]
df_f7_hadm = df_f7[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n7 = (np.shape(df_f7_hadm)[0])
n7

42865

In [48]:
df_f7_hadm.head()

,subject_id,hadm_id,long_title,icd_code,icd_version
886,16569548,24870770,"congestive heart failure, unspecified",4280,9
1050,18460230,25771608,"congestive heart failure, unspecified",4280,9
2342,19661870,22135027,"congestive heart failure, unspecified",4280,9
2683,18932584,23973570,"congestive heart failure, unspecified",4280,9
2961,16723797,28913496,"congestive heart failure, unspecified",4280,9


## Part III: DRG codes

In [49]:
drgcodes = pd.read_csv(f'{mimiciv_dir}/drgcodes.csv', low_memory=False)

In [50]:
# merge tables
df_ids_icds = pd.merge(df_event, df_code, on = ['icd_code','icd_version'], how='left')
df_merged = pd.merge(df_ids_icds, drgcodes, on = ['hadm_id', 'subject_id'], how='left')
df_merged = df_merged.drop(columns=['drg_severity', 'drg_mortality'])

In [51]:
# drop any rows that don't have a drg_code
df_merged = df_merged.dropna(subset=['drg_code'], how='any')

In [52]:
# select rows where DRG code contains "hf" or "heart failure"
df_sub = df_merged[df_merged['description'].str.contains('hf|heart failure', case=False)]
# there are some drg codes that specify "w/o hf" or "w/o heart failure" - don't include those rows
df_sub = df_sub[~df_sub['description'].str.contains('W/O AMI/HF/SHOCK|W/O AMI, HEART FAILURE OR SHOCK', case=False)]

In [54]:
# remove the hadm_ids that have a long title with heart failure in them, since the ICD codes already account for them
df_sub = df_sub.dropna(subset=['icd_code','icd_version','hadm_id','subject_id','long_title'], how='any')

no_hf_icd = []
for hadm_id in df_sub.hadm_id.unique():
    df_hadm_id = df_sub[df_sub['hadm_id'] == hadm_id]
    if len(df_hadm_id[df_hadm_id['long_title'].str.contains('heart failure', case=False)]) == 0:
        no_hf_icd.append(hadm_id)
df_no_hf_icd = df_sub[df_sub['hadm_id'].isin(no_hf_icd)]

In [55]:
# only keep rows where ICD codes include:
#" cardiogenic shock", "left ventricular failure, unspecified", "shock, unspecified", or pleural effusion or fluid overload
df_no_hf_icd = df_no_hf_icd[df_no_hf_icd['long_title'].str.contains('cardiogenic shock|left ventricular failure, unspecified|shock, unspecified|Pleural effusion, not elsewhere classified|Fluid overload, unspecified', case=False)]
df_no_hf_icd_hadm = df_no_hf_icd[['subject_id','hadm_id','long_title', 'icd_code','icd_version', 'description']].drop_duplicates(subset=['hadm_id'])

## Part IV: IV Diuretics

In [78]:
chf_hadm = df_f7_hadm.hadm_id.append(df_no_hf_icd_hadm.hadm_id).unique() # 51322 hadm
len(chf_hadm)

42883

In [58]:
df_diuretics = pd.DataFrame({'furosemide': (df_emar['medication'].str.contains('furosemide', case=False, na=False) | df_emar['medication'].str.contains('lasix', case=False, na=False)) & ~df_emar['medication'].str.contains('Desensitization', case=False, na=False), 
                   'bumetanide': (df_emar['medication'].str.contains('bumetanide', case=False, na=False) | df_emar['medication'].str.contains('bumex', case=False, na=False)) & ~df_emar['medication'].str.contains('Desensitization', case=False, na=False), 
                   'chlorothiazide': (df_emar['medication'].str.contains('chlorothiazide', case=False, na=False) | df_emar['medication'].str.contains('diuril', case=False, na=False)) & ~df_emar['medication'].str.contains('Desensitization', case=False, na=False) & ~df_emar['medication'].str.contains('hydrochlorothiazide', case=False, na=False)})

In [59]:
# prescriptions for diuretics
given_diuretic = df_diuretics.any(axis=1)

In [104]:
# Determine which meds are given by IV (route_iv_detail) and which are given by another route (route_not_iv_detail)
route_iv_detail = df_emar_detail.route.str.contains('IV', case=False, na=False)
route_not_iv_detail = ~df_emar_detail.route.str.contains('IV', case=False, na=True)



NameError: name 'df_emar_details' is not defined

In [112]:
# Convert above values to have one value per emar
from collections import defaultdict
num_emar_details = df_emar_detail.shape[0]
route_iv = defaultdict(lambda:False)
route_not_iv = defaultdict(lambda:False)

for i in range(num_emar_details):
    emar_id = df_emar_detail.emar_id[i]
    route_iv[emar_id] = route_iv[emar_id] or route_iv_detail[i]
    route_not_iv[emar_id] = route_not_iv[emar_id] or route_not_iv_detail[i]
    
    if i % 100000 == 0:
        print(i)
        

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [115]:
route_iv_idx = np.array([not route_not_iv[emar_id] for emar_id in df_emar.emar_id.to_numpy()])
print(route_iv_idx.sum())

19269990


In [94]:
# prescriptions with hadm_id associated w heart failure ICD
in_icd = np.isin(df_emar.hadm_id, chf_hadm)


In [116]:
# final cohort with all prescription fields
cohort = df_emar[given_diuretic & in_icd & route_iv_idx]

In [117]:
# unique drug names to confirm no unwanted drugs are included
cohort.medication.unique()

array(['Furosemide', 'Furosemide-Heart Failure', 'Chlorothiazide',
       'Bumetanide', 'Furosemide in 0.9% Sodium Chloride', 'furosemide'],
      dtype=object)

In [118]:
cohort.hadm_id.unique().shape[0] #25691 unique HADM_IDs

11614

In [119]:
cohort.subject_id.unique().shape[0] #14757 unique SUBJECT_IDs

7158

In [120]:
cohort_hadm = cohort.hadm_id.unique()

## Part V: Removing Dialysis Patients 

In [121]:
dialysis_codes = ["Z992", "N186", "5856", "V4511"]

In [122]:
df_event["chronic_dialysis"] = df_event["icd_code"].str.strip().isin(dialysis_codes)

In [123]:
dialysis_hadm = df_event[df_event["chronic_dialysis"] == True].hadm_id

In [124]:
df_event["chronic_dialysis"] = df_event['hadm_id'].isin(dialysis_hadm)

## Part VI: Final Cohort

In [125]:
final_cohort = df_event.drop(columns=['seq_num', 'icd_code', 'icd_version'])
final_cohort.head()

,subject_id,hadm_id,chronic_dialysis
0,11603789,25552978,False
1,11603789,25552978,False
2,11603789,25552978,False
3,11603789,25552978,False
4,11603789,25552978,False


In [126]:
final_cohort['CHF exacerbation'] = final_cohort['hadm_id'].isin(cohort_hadm)

In [127]:
final_cohort = final_cohort.drop_duplicates()

In [128]:
final_cohort.head()

,subject_id,hadm_id,chronic_dialysis,CHF exacerbation
0,11603789,25552978,False,False
5,16809467,29803749,False,False
10,11147319,25941345,False,False
14,12156531,29321418,False,False
19,14528388,29429446,False,False


In [129]:
len(final_cohort) #536165

443067

In [130]:
np.sum(final_cohort.hadm_id.value_counts() > 1)

0

In [131]:
final_cohort.to_csv('final_cohort.csv')

In [132]:
sum(final_cohort['chronic_dialysis']==False)

430281

In [133]:
final_cohort_selected = final_cohort[(final_cohort['chronic_dialysis']==False) & (final_cohort['CHF exacerbation']==True)]
final_cohort_selected.head()
final_cohort_selected.to_csv('final_cohort_selected.csv')

In [134]:
len(final_cohort_selected)

11213